In [1]:
import pandas as pd
from pymystem3 import Mystem
df = pd.DataFrame()
m=Mystem()
for i in range(1,46):
    if(i in [9,22,26]): continue
    df2 = pd.read_json('Texts/art{}syn.json'.format(i))
    df2['file'] = i
    df = pd.concat([df,df2]) 


df = df.reset_index()
def get_struct_data(x):
    sentl =[]
    pars=[]
    for text in x:
        for sent in text:
            sentl = sent.split(' ')
            to_app = [sentl[i] for i in range(len(sentl)) if i in([0,1,4,6,8,10,12])]
            to_app.append(m.lemmatize(to_app[2])[0])
            pars.append(to_app)
    return(pars)
       
df['syntax'] = df['syntax'].apply(get_struct_data)  


def get_mutual_node(n_file,row,ent1,ent1pos,ent2,ent2pos):
    syntax = (df[(df['file']==n_file) & (df['index']==row)].syntax.iloc[0])
    
    def get_node(e,p):
        e = e.split(" ")[0]
        ent_dict = dict()
        for i in range(len(syntax)):
            if(syntax[i][7]==e):
                ent_dict[abs(int(syntax[i][1])-p)]=i
        return ent_dict[sorted(ent_dict)[0]]
    def get_route(i):
        route = list()
        for _ in range(len(syntax)):
            h=int(syntax[i][5])
            if(h ==-1): return route
            route.append(syntax[h])
            i=h
    ent_node1 = get_node(ent1,ent1pos)
    ent_node2 = get_node(ent2,ent2pos)
    route1= get_route(ent_node1)[::-1]
    route2= get_route(ent_node2)[::-1]
    
    for i in range(len(route1)):
        if(route1[i][0]==route2[i][0]): obj=route1[i]
        else: break
    
    return obj,syntax[ent_node1],syntax[ent_node2]
    

In [2]:
get_mutual_node(1,3,'сирия',2,'румийлан',2)

(['31',
  '42',
  'перебросить',
  'VERB',
  'Aspect=Perf|VerbForm=Inf|fPOS=VERB++',
  '2',
  'nsubj',
  'перебрасывать'],
 ['78',
  '83',
  'Сирии',
  'NOUN',
  'Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing|fPOS=NOUN++',
  '10',
  'dobj',
  'сирия'],
 ['107',
  '115',
  'Румийлан',
  'NOUN',
  'Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing|fPOS=NOUN++',
  '15',
  'appos',
  'румийлан'])